In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon, LineString, mapping, Point
import numpy as np
import itertools
import rtree
from tqdm import tqdm
pd.set_option('max_columns', None)
%matplotlib inline


street_segments = gpd.read_file("../data/processed/street_segments.geojson")
blocks = gpd.read_file("../data/processed/blocks.geojson")
buildings = gpd.read_file("../data/processed/buildings.geojson")
blockfaces = gpd.read_file("../data/processed/blockfaces.geojson")

In [2]:
def filter_on_block_id(block_id, block_id_key="geoid10"):
    def select(df):
        return (df.set_index(block_id_key)
                    .filter(like=block_id, axis='rows')
                    .reset_index()
               )
    
    return select


def get_block_data(block_id):
    ss = street_segments.pipe(filter_on_block_id(block_id))
    bf = blockfaces.pipe(filter_on_block_id(block_id))
    bldgs = buildings.pipe(filter_on_block_id(block_id))
    return ss, bf, bldgs


def plot_block(block_id):
    street_segments, blockfaces, buildings = get_block_data(block_id)
    
    ax = street_segments.plot(color='red', linewidth=1)
    blockfaces.plot(color='black', ax=ax, linewidth=1)
    buildings.plot(ax=ax, color='lightsteelblue', linewidth=1, edgecolor='steelblue')
    return ax


def simplify_linestring(inp):
    inp = inp.convex_hull
    coords = np.round(mapping(inp)['coordinates'], decimals=4)
    out = LineString(coords)
    return out
    
    
def simplify_bldg(bldg):
    if isinstance(bldg, MultiPolygon):
        bldg = bldg.buffer(0)
    
    if isinstance(bldg, MultiPolygon):
        raise NotImplemented  # TODO
    
    coords = [xyz[:2] for xyz in mapping(bldg.convex_hull)['coordinates'][0]]
    bldg = Polygon(bldg)
    
    return bldg


def pairwise_combinations(keys):
    return itertools.combinations({'a': 12, 'b': 15, 'c': 13}.keys(), r=2)


def collect_strides(point_observations):
    point_obs_keys = list(point_observations.keys())
    curr_obs_start_offset = point_obs_keys[0]
    curr_obs_start_bldg = point_observations[point_obs_keys[0]]    
    strides = dict()
    
    for point_obs in point_obs_keys[1:]:
        bldg_observed = point_observations[point_obs]
        if bldg_observed != curr_obs_start_bldg:
            strides[(curr_obs_start_offset, point_obs)] = curr_obs_start_bldg
            curr_obs_start_offset = point_obs
            curr_obs_start_bldg = bldg_observed
        else:
            continue
            
    strides[(curr_obs_start_offset, '1.00')] = bldg_observed
    
    return strides


def get_stride_boundaries(strides, step_size=0.02):
    boundaries = list()
    
    keys = list(strides.keys())
    for idx, key in enumerate(keys[1:]):
        curr = strides[key]
        boundaries.append((key[0], str(float(key[0]) + step_size)))

    return boundaries


def cut(line, distance):
    # Cuts a line in two at a distance from its starting point
    if distance <= 0.0 or distance >= 1.0:
        return [LineString(line)]
    coords = list(line.coords)
    for i, p in enumerate(coords):
        pd = line.project(Point(p), normalized=True)
        if pd == distance:
            return [
                LineString(coords[:i+1]),
                LineString(coords[i:])]
        if pd > distance:
            cp = line.interpolate(distance, normalized=True)
            return [
                LineString(coords[:i] + [(cp.x, cp.y)]),
                LineString([(cp.x, cp.y)] + coords[i:])]


def reverse(l):
    l_x, l_y = l.coords.xy
    l_x, l_y = l_x[::-1], l_y[::-1]
    return LineString(zip(l_x, l_y))


def frontages_for_blockface(bldgs, blockface, step_size=0.01):
    index = rtree.Rtree()
    
    if len(bldgs) == 0:
        return gpd.GeoDataFrame()
    
    for idx, bldg in bldgs.iterrows():
        index.insert(idx, bldg.geometry.bounds)
    
    bldg_frontage_points = dict()
    
    search_space = np.arange(0, 1, step_size)
    next_search_space = []
    while len(search_space) > 0:
        for offset in search_space:
            search_point = blockface.geometry.interpolate(offset, normalized=True)
            nearest_bldg = list(index.nearest(search_point.bounds, 1))[0]
            bldg_frontage_points[str(offset)[:6]] = nearest_bldg
            
        strides = collect_strides(bldg_frontage_points)
        search_space = next_search_space
        
    # convert the list of strides to a proper GeoDataFrame
    out = []
    for sk in strides.keys():
        srs = bldgs.loc[strides[sk], ['geoid10', 'sf16_BldgID']]
        srs['geoid10_n'] = blockface['geoid10_n']
        srs['geom_offset_start'] = sk[0]
        srs['geom_offset_end'] = sk[1]
        out.append(srs)
    
    out = gpd.GeoDataFrame(out)
    
    geoms = chop_line_segment_using_offsets(blockface.geometry, strides)
    out['geometry'] = geoms

    return out

In [3]:
# model_street_segments, model_blockfaces, model_buildings = get_block_data('060759809001134')

In [4]:
# frontages_for_blockface(model_buildings, model_blockfaces.iloc[0]).plot(color=['red', 'green', 'blue'])

In [3]:
from tqdm import tqdm

In [4]:
def calculate_frontages(blocks, streets, blockfaces, buildings):
    frontages = []

    for block_idx, block in tqdm(list(blocks.iterrows())):
        _, blockfaces, buildings = get_block_data(block.geoid10)
        for blockface_idx, blockface in blockfaces.iterrows():
            result = frontages_for_blockface(buildings, blockface)
            frontages.append(result)
    
    frontages = gpd.GeoDataFrame(pd.concat(frontages)).groupby('sf16_BldgID').apply(
        lambda df: df.assign(sf16_BldgID_n=[f'{df.iloc[0].sf16_BldgID}_{n}' for n in range(len(df))])
    ).reset_index(drop=True)
    
    return frontages

In [51]:
# frontages = calculate_frontages(blocks.iloc[:1000], street_segments, blockfaces, buildings)

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import garbageman

In [15]:
out = garbageman.pipeline.calculate_frontages(blocks.iloc[121:122], street_segments, blockfaces, buildings)





  0%|          | 0/1 [00:00<?, ?it/s]



100%|██████████| 1/1 [00:01<00:00,  1.41s/it]





TODO: check frontages for consistency.